<a href="https://colab.research.google.com/github/UdithWeerasinghe/IntelliScript_phase02_BIG/blob/main/EX7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install pandas openpyxl

In [ ]:
# Import necessary libraries
import pandas as pd
import os
import json
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

def process_excel_to_timeseries(input_directory, output_directory):
    """
    Processes all Excel files in the input directory, extracts time series data, and saves them to the output directory.

    Parameters:
        input_directory (str): Path to the input folder in Google Drive containing Excel files.
        output_directory (str): Path to the output folder in Google Drive to save JSON and Excel files.

    Returns:
        None
    """
    os.makedirs(output_directory, exist_ok=True)
    consolidated_data = []

    # Iterate through all files in the input directory
    for file_name in os.listdir(input_directory):
        if file_name.endswith(('.xlsx', '.xls')):
            file_path = os.path.join(input_directory, file_name)
            print(f"Processing file: {file_name}")

            # Read the Excel file
            xls = pd.ExcelFile(file_path)
            for sheet_name in xls.sheet_names:
                print(f"  Processing sheet: {sheet_name}")
                df = pd.read_excel(xls, sheet_name=sheet_name)

                if 'Date' not in df.columns:
                    print(f"  Skipping sheet '{sheet_name}' as it does not contain a 'Date' column.")
                    continue

                # Clean and format the data
                df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
                df = df.dropna(subset=['Date'])  # Drop rows with invalid dates
                df = df.sort_values(by='Date')  # Sort by date

                # Prepare JSON structure
                time_series_data = {}
                for col in df.columns:
                    if col != 'Date':
                        time_series_data[col] = [
                            {"Date": row['Date'].isoformat(), "value": row[col]} for _, row in df[['Date', col]].dropna().iterrows()
                        ]

                # Save individual JSON file for each sheet
                json_output_path = os.path.join(output_directory, f"{file_name}_{sheet_name}.json")
                with open(json_output_path, 'w') as json_file:
                    json.dump(time_series_data, json_file, indent=4)
                print(f"    Saved JSON file: {json_output_path}")

                # Consolidate data for combined Excel output
                df.rename(columns={"Date": "Date"}, inplace=True)
                consolidated_data.append(df)

    # Save consolidated Excel file
    if consolidated_data:
        combined_df = pd.concat(consolidated_data, ignore_index=True)
        excel_output_path = os.path.join(output_directory, "consolidated_timeseries.xlsx")
        combined_df.to_excel(excel_output_path, index=False)
        print(f"Saved consolidated Excel file: {excel_output_path}")
    else:
        print("No valid data found to save.")






In [ ]:
# Define Google Drive paths
input_folder = "/content/drive/MyDrive/TSD"  # Replace with your input folder path
output_folder = "/content/drive/MyDrive/JSD"  # Replace with your output folder path

In [ ]:
# Execute the ETL pipeline
process_excel_to_timeseries(input_folder, output_folder)

Processing file: EXPORTS- LKR Millions.xlsx
  Processing sheet: Sheet1
    Saved JSON file: /content/drive/MyDrive/JSD/EXPORTS- LKR Millions.xlsx_Sheet1.json
Processing file: IMPORTS - LKR Millions.xlsx
  Processing sheet: Sheet1
    Saved JSON file: /content/drive/MyDrive/JSD/IMPORTS - LKR Millions.xlsx_Sheet1.json
Saved consolidated Excel file: /content/drive/MyDrive/JSD/consolidated_timeseries.xlsx
